# Benchmarking SQL vs Python

The work we have done so far is interesting but largely academic. The question remains as to why we may want to do these kinds of matrix calculations inside of SQL instead of loading the matrices into Python and using the mature libraries (such as NumPy) that exist for exactly such calculations. In this notebook, we investigate the relative speed of both methods for large random matrices. We also investigate how our results may be affected by slow internet speeds by using simulated delay. 

## Initial Setup

### Variables

In [1]:
INTERNET_SPEED = 1000 # The simulated internet speed in Mbps
DIGIT_BITS = 64
DELAY =  False

In [2]:
# Matrix sizes:
#   A is n*m
#   B is m*p
n = 10
m,p = n,n 

### Load SQL

In [3]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

In [4]:
load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL")
engine = create_engine(DATABASE_URL)

#### Quick test

In [5]:
with engine.connect() as con:
    result = con.execute(text("SELECT * FROM A")).fetchall()

    for matrix in result:
        print(matrix)

(1, 1, 1.0)
(1, 2, 2.0)
(1, 3, 0.0)
(2, 1, 1.0)
(2, 2, 1.0)
(2, 3, 4.0)
(3, 1, 0.0)
(3, 2, 7.0)
(3, 3, 1.0)


### Generate Matrices

In [6]:
import numpy as np

In [7]:
def put_mat(n:int, m:int, name:str, mat):
    sql = f"""INSERT INTO {name} (i, j, value) VALUES"""
    for i in range(n):
        for j in range(m):
            if i==1 and j==1:
                sql += f"({i+1}, {j+1}, {mat[i][j]}),"
            else:
                sql += f"\n({i+1}, {j+1}, {mat[i][j]}),"
    sql= sql[:-1] + ";"
    with engine.connect() as con:
        con.execute(text(f"DROP TABLE IF EXISTS {name};"))
        con.execute(text(f"""
CREATE TABLE {name} (
    i INT NOT NULL,
    j INT NOT NULL,
    value DOUBLE PRECISION NOT NULL,
    PRIMARY KEY (i, j)
);"""))
        con.execute(text(sql))
        con.commit()


In [8]:
def gen_mat(n:int, m:int, name:str) -> None:
    random = np.random.rand(n,m)
    put_mat(n,m,name,random)


### Get Matrices into Python

In [9]:
def get_mat(n:int, m:int, name:str):
    mat = np.zeros((n,m))
    with engine.connect() as con:
        result = con.execute(text(f"SELECT * FROM {name}")).fetchall()
        for i,j,value in result:
            mat[i-1][j-1] = value
    return mat

In [10]:
import time

In [11]:
def delay(bits, speed=INTERNET_SPEED):
    delay = bits/speed
    time.sleep(delay)

## Using Python

In [12]:
def python_test(n, m, p):
    start = time.time()
    mat1 = get_mat(n,m,"mat1")
    if DELAY:
        delay(DIGIT_BITS*m*(n+p),INTERNET_SPEED)
    mat2 = get_mat(m,p,"mat2")
    mat3 = np.dot(mat1, mat2)
    put_mat(n, p, "mat3_py",mat3)
    end = time.time()
    return end - start, mat3
    

## Using SQL

In [13]:
def sql_test(n, p):
    start = time.time()
    with engine.connect() as con:
        con.execute(text(f"SELECT mat_mul('mat1','mat2','mat3_sql');"))
        con.commit()
    end = time.time()
    return end-start, get_mat(n,p,"mat3_sql")

## Putting it all together

In [14]:
def test(n:int, m:int,p:int):
    gen_mat(n, m, "mat1")
    gen_mat(m, p, "mat2")
    py_time, py_mat3 = python_test(n, m, p)
    sql_time, sql_mat3 = sql_test(n, p)
    assert np.all(np.isclose(sql_mat3, py_mat3))
    return py_time-sql_time


### Drawing some graphs

In [ ]:
xs = np.arange(100, 1000, 100)
ys = []
for x in xs:
    ys.append(test(x,x,x))


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(xs, ys)
plt.xlabel("Matrices are of dimension n")
plt.ylabel("Time difference between SQL and Python approach, >0 means SQL wins (s)")
if DELAY:
    plt.title("Time difference between SQL and Python with delay")
else:
    plt.title("Time difference between SQL and Python with no delay")
plt.show()